# Twitter Images

Twitterで画像を取得し，それをデータセットとして保管するための仕組みを構築する．

## データベースの構築

sqliteを使用してデータの再読み込みやキャッシュとしての保管を行う．

In [ ]:
import sqlite3
from datetime import datetime
import time

def adapt_datetime(ts):
    return time.mktime(ts.timetuple())

dbname = ('dataset.db')
conn = sqlite3.connect(dbname)
cur = conn.cursor()

jupyter上で触ることができるようにする．

In [ ]:
%load_ext sql
%sql sqlite:///dataset.db
    
%sql select name from sqlite_master where type='table'

以下のテーブルを作成する．
- `tweets` ツイートに関するテーブル
- `twitter_accounts` 探索したツイッターアカウントに関するテーブル
- `twitter_images` 探索した画像に関するテーブル
- `twitter_hashtags` ハッシュタグに関するテーブル

In [ ]:
%%sql

-- DROP TABLE twitter_accounts;
-- DROP TABLE twitter_images;
-- DROP TABLE twitter_hashtags;
-- DROP TABLE tweets;
-- DROP TABLE image_tweets;
-- DROP TABLE tweet_hashtags;
SELECT name FROM sqlite_master WHERE type='table';

In [ ]:
%%sql

CREATE TABLE if not exists test (
    id integer PRIMARY KEY
);

-- twitter_accounts table
CREATE TABLE if not exists twitter_accounts (
    account_id integer PRIMARY KEY,
    twitter_id char[32] not null UNIQUE, -- unique identifier ex: "2244994945"
    checked_at datetime not null,
    twitter_name char[32] not null,
    num_follows int not null, -- if it's -1, means account deleted
    num_followers int not null,
    num_listed int not null
);

-- twitter_images table
CREATE TABLE if not exists twitter_images (
    image_id integer PRIMARY KEY,
    twitter_id char[256] not null UNIQUE,
    twitter_url char[256] not null,
    file_path char[256] not null UNIQUE,
    sha256 char[64], -- if it's null, we haven't downloaded yet
    width integer not null,
    height integer not null
);

-- hashtags table
CREATE TABLE if not exists twitter_hashtags (
    hashtag_id integer PRIMARY KEY,
    hashtag char[256] not null UNIQUE
);

-- languages table
CREATE TABLE if not exists twitter_languages (
    language_id integer PRIMARY KEY,
    language char[16] not null UNIQUE
);

-- tweets table
CREATE TABLE if not exists tweets (
    tweet_id integer PRIMARY KEY,
    twitter_tweet_id char[256] not null UNIQUE,
    account_id integer not null,
    language_id integer not null,
    created_at datetime not null,
    checked_at datetime not null,
    num_likes integer not null,
    num_retweets integer not null,
    num_replys integer not null
);

-- image tweet pairs table
CREATE TABLE if not exists tweet_images (
    tweet_id integer,
    image_id integer
);

-- tweet hashtag pairs table
CREATE TABLE if not exists tweet_hashtags (
    tweet_id integer,
    hashtag_id integer
);

DROP TABLE test;
SELECT name FROM sqlite_master WHERE type='table';

## ツイートと画像のセットの取得

In [ ]:
from urllib3 import PoolManager
import urllib
import os
import json
from datetime import datetime

### CONSTANTS
headers = {'Authorization': 'Bearer ' + os.getenv("TWITTER_TOKEN")}
search_word = "#絵描きさんと繋がりたい -is:retweet -is:reply"
num_result = 100
http = PoolManager()

search_url = 'https://api.twitter.com/2/tweets/search/recent'
params = {
    'query': search_word,
    'expansions': 'author_id,attachments.media_keys',
    'media.fields': 'type,url,width,height',
    'tweet.fields': 'id,created_at,lang,text,entities,public_metrics',
    'user.fields': 'id,name,username,public_metrics',
    'max_results': num_result,
#     'next_token': 'b26v89c19zqg8o3fpe18pf6fkq7vf2rsr1jxzbexd0e4d',
}

req = http.request('GET',
                    search_url,
                    headers=headers,
                    fields=params)

mapping = json.loads(req.data)


next_token
- "#絵師さんと繋がりたい -is:retweet -is:reply"
    - b26v89c19zqg8o3fpe18pav6pjgxiiji2d8pxrfte2zjx
- "#絵柄が好みって人にフォローされたい -is:retweet -is:reply"
    - b26v89c19zqg8o3fpe18pf4xvov2kk6t7xet0ipi8evzx
- "#絵描きさんと繋がりたい -is:retweet -is:reply"
    - b26v89c19zqg8o3fpe18phb5mzuzuqecxpnigtcrortrx
- "#イラスト -is:retweet -is:reply"
    - b26v89c19zqg8o3fpe18ph9nqew8ez2vdknjkqkxzoarh

In [ ]:
mapping['meta']

In [ ]:
# accounts
accounts = {account['id']:{
    'twitter_id': account['id'],
    'num_follows': account['public_metrics']['following_count'],
    'num_followers': account['public_metrics']['followers_count'],
    'num_listed': account['public_metrics']['listed_count'],
    'username': account['username'],
} for account in mapping['includes']['users']}

In [ ]:
# make directory
import os
for _, account in accounts.items():
    if os.path.isdir("./images/"+account['username']) == False:
        os.mkdir("./images/"+account['username'])

# register to database
args = [(item['twitter_id'], item['username'], 
         item['num_follows'], item['num_followers'], item['num_listed'])
       for _, item in accounts.items()]

cur.executemany(f"""
INSERT INTO twitter_accounts
    (twitter_id, checked_at, twitter_name, num_follows, num_followers, num_listed)
VALUES(?, datetime('now', 'utc'), ?, ?, ?, ?)
ON CONFLICT(twitter_id)
    DO UPDATE SET
        checked_at = excluded.checked_at,
        twitter_name = excluded.twitter_name,
        num_follows = excluded.num_follows,
        num_followers = excluded.num_followers,
        num_listed = excluded.num_listed
""", args)

conn.commit()

In [ ]:
%%sql
select COUNT(*) from twitter_accounts
    

In [ ]:
import re

images = {
    media['media_key']:{
        'url':media['url'],
        'width':media['width'],
        'height':media['height'],
    }
    for media in mapping['includes']['media']
        if media['type'] == 'photo'
}

for tweet in mapping['data']:
    if 'attachments' in tweet:
        for media_key in tweet['attachments']['media_keys']:
            if media_key in images:
                author = accounts[tweet['author_id']]['username']
                url = images[media_key]['url']
                images[media_key]['author'] = author
                images[media_key]['path'] = "./images/"+author+"/"+re.findall(r'[^/]+$', url)[0]

if sum([1 if 'author' not in image else 0 for _, image in images.items()]) > 0:
    raise('error occered: we have some images unknown writer')
    


In [ ]:
import hashlib

for media_id, image in images.items():
    try:
        urllib.request.urlretrieve(image['url'], image['path'])
        hash = hashlib.sha256()
        
        with open(image['path'], 'rb') as f:
            chunk = f.read(2048 * hash.block_size)
            if len(chunk) == 0:
                break
            hash.update(chunk)
        
        args = (media_id, image['url'], image['path'], hash.hexdigest(), image['width'], image['height'])
        cur.execute("""
            INSERT INTO twitter_images
                (twitter_id, twitter_url, file_path, sha256, width, height)
            VALUES (?, ?, ?, ?, ?, ?)
            ON CONFLICT(twitter_id)
                DO UPDATE SET sha256 = excluded.sha256
        """, args)
    except e:
        print(e)
        args = (media_id, image['url'], image['path'], image['width'], image['height'])
        cur.execute("""
            INSERT INTO twitter_images
                (twitter_id, twitter_url, file_path, width, height)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(twitter_id) DO NOTHING
        """, args)

conn.commit()

In [ ]:
from dateutil.parser import isoparse

tweets = {tweet['id']:{
    'author_id':tweet['author_id'],
    'created_at': isoparse(tweet['created_at']),
    'num_likes':tweet['public_metrics']['like_count'],
    'num_retweets':tweet['public_metrics']['retweet_count'],
    'num_replys':tweet['public_metrics']['reply_count'],
    'language': tweet['lang'],
    'hashtags':([] if 'hashtags' not in tweet['entities']
               else [hashtag['tag'] for hashtag in tweet['entities']['hashtags']]),
    'image_id':([] if ('attachments' not in tweet) or ('media_keys' not in tweet['attachments'])
             else [media_key for media_key in tweet['attachments']['media_keys'] if media_key in images])
}
          for tweet in mapping['data']}

hashtags = set()
languages = set()
for _, tweet in tweets.items():
    hashtags |= set(tweet['hashtags'])
    languages.add(tweet['language'])

In [ ]:
args = [(hashtag,) for hashtag in hashtags]

cur.executemany("""
INSERT INTO twitter_hashtags (hashtag)
VALUES(?)
ON CONFLICT(hashtag) DO NOTHING
""", args)
conn.commit()

args = [(language,) for language in languages]
cur.executemany("""
INSERT INTO twitter_languages (language)
VALUES(?)
ON CONFLICT(language) DO NOTHING
""", args)
conn.commit()

In [ ]:
args = [(tweet_id,
         tweet['author_id'],tweet['language'],
         tweet['created_at'],tweet['num_likes'],
         tweet['num_retweets'], tweet['num_replys'])
       for tweet_id, tweet in tweets.items()]

cur.executemany("""
INSERT INTO tweets(
    twitter_tweet_id, account_id, language_id, 
    created_at, checked_at, 
    num_likes, num_retweets, num_replys)
VALUES (
    ?,
    (SELECT twitter_accounts.account_id 
        FROM twitter_accounts
        WHERE twitter_accounts.twitter_id = ?),
    (SELECT twitter_languages.language_id
        FROM twitter_languages
        WHERE twitter_languages.language = ?),
    ?,
    datetime('now','utc'),
    ?,?,?
) ON CONFLICT(twitter_tweet_id)
    DO UPDATE SET
        checked_at = excluded.checked_at,
        num_likes = excluded.num_likes,
        num_retweets = excluded.num_retweets,
        num_replys = excluded.num_replys
""", args)

conn.commit()

In [ ]:
%%sql
SELECT COUNT(*) from tweet_accounts;

In [ ]:
args = [(
    image_id, tweet_id
) for tweet_id, tweet in tweets.items() for image_id in tweet['image_id']]

cur.executemany("""
INSERT INTO tweet_images (image_id, tweet_id)
SELECT 
    (SELECT twitter_images.image_id
        FROM twitter_images
        WHERE twitter_images.twitter_id = ?) AS tmp_image_id,
    (SELECT tweets.tweet_id
        FROM tweets
        WHERE tweets.twitter_tweet_id = ?) AS tmp_tweet_id
WHERE NOT EXISTS (
    SELECT * FROM tweet_images AS counting
        WHERE (counting.tweet_id = tmp_tweet_id AND
            counting.image_id = tmp_image_id)
)
""", args)

conn.commit()

In [ ]:
%%sql
SELECT COUNT(*) from tweet_images;

In [ ]:
args = [(
    tweet_id, hashtag
) for tweet_id, tweet in tweets.items() for hashtag in tweet['hashtags']]

cur.executemany("""
INSERT INTO tweet_hashtags (tweet_id, hashtag_id)
SELECT 
    (SELECT tweets.tweet_id
        FROM tweets
        WHERE tweets.twitter_tweet_id = ?) AS tmp_tweet_id,
    (SELECT twitter_hashtags.hashtag_id
        FROM twitter_hashtags
        WHERE twitter_hashtags.hashtag = ?) AS tmp_hashtag_id
WHERE NOT EXISTS (
    SELECT * FROM tweet_hashtags AS counting
        WHERE (counting.tweet_id = tmp_tweet_id AND
            counting.hashtag_id = tmp_hashtag_id)
)
""", args)

conn.commit()

In [ ]:
%%sql
SELECT hashtag, num_tweets, num_images
FROM twitter_hashtags
JOIN ( -- num_tweets
    SELECT hashtag_id, COUNT(hashtag_id) AS num_tweets FROM tweet_hashtags
        GROUP BY hashtag_id
) AS count_tweets ON twitter_hashtags.hashtag_id = count_tweets.hashtag_id
JOIN ( -- num_images
    SELECT hashtag_id, COUNT(hashtag_id) AS num_images FROM tweet_images
        JOIN (
            SELECT * FROM tweet_hashtags
        ) AS count_images ON tweet_images.tweet_id = count_images.tweet_id
        GROUP BY hashtag_id
) AS count_tweet_images ON twitter_hashtags.hashtag_id = count_tweet_images.hashtag_id
ORDER BY num_images DESC
LIMIT 20

In [ ]:
%%sql
SELECT num_images, num_tweets, COUNT(*) AS count_users FROM (
    SELECT 
        twitter_accounts.twitter_name, 
        twitter_accounts.account_id, 
        num_tweets, 
        num_images
    FROM twitter_accounts
    JOIN ( -- num_tweets
        SELECT account_id, COUNT(account_id) AS num_tweets FROM tweets
            GROUP BY account_id
    ) AS count_tweets ON twitter_accounts.account_id = count_tweets.account_id
    JOIN ( -- num_images
        SELECT account_id, COUNT(account_id) AS num_images 
        FROM tweet_images
        JOIN (
            SELECT account_id, tweet_id FROM tweets
        ) AS count_tweet_images ON tweet_images.tweet_id = count_tweet_images.tweet_id
            GROUP BY account_id
    ) AS count_images ON twitter_accounts.account_id = count_images.account_id
)
GROUP BY num_images, num_tweets

In [ ]:
%%sql
